In [ ]:
import json
import re
import os
import openai
import time

In [ ]:
data_path = ""

tttt_inputs = []
for idx, file in enumerate(os.listdir(data_path)):
    if file.endswith(".txt"):
        with open(os.path.join(data_path, file), "r") as f:
            tttt_input = f.readlines()
        tttt_inputs.append((file[:-4], ''.join(tttt_input)))

In [ ]:
def create_game_json(tttt_input):

    snippets = tttt_input[1].split("\n\n")

    game_json = {}
    game_json['Episode'] = tttt_input[0]
    conversation = []

    appended = 0
    for snip in snippets:
        if snip.lower().startswith("name"):
            game_json['Name'] = snip.split("Name:")[1].lstrip(" \n").rstrip(" \n")
            appended += 1

        if snip.lower().startswith("affidavit"):
            game_json['Affidavit'] = snip.split("Affidavit:")[1].lstrip(" \n").rstrip(" \n")
            appended += 1

        if snip.lower().startswith('conversations'):
            conversation.append(snip.split("Conversations:")[1].lstrip(" \n").rstrip(" \n"))
            appended += 1

        if snip.startswith("Q:"):
            if "[R]" in snip:
                snip = snip.replace("[R]", "")
                try:
                    snip = conversation[-1] + "\n" + snip
                    conversation.append(snip)
                    appended += 1
                except:
                    conversation.append(snip)
                    appended += 1
            else:
                conversation.append(snip)
                appended += 1

    game_json['Conversation'] = conversation

    if len(snippets) > appended:
        print("EP: {}, Total: {}, App: {}".format(tttt_input[0], len(snippets), appended))
    
    return game_json

In [ ]:
all_game_jsons = {}
for tttt_input in tttt_inputs:
    all_game_jsons[tttt_input[0]] = create_game_json(tttt_input)

In [ ]:
from collections import defaultdict
import pandas as pd

gold_answers = pd.read_csv("", encoding='utf-8')
gold_answers_map = {}
answer_map = defaultdict(str, {'1': "number one", '2': "number two", '3': "number three"})
for _, row in gold_answers.iterrows():
    gold_answers_map["e{}s{}".format(str(row['episode']), str(row['session']))] = answer_map[str(row['cc'])]

In [ ]:
openai.api_key = ""

accuracies = []
for key, game in all_game_jsons.items():

    new_messages = [
            {"role": "system",
            "content": "You are an expert detective."
            },
        ]

    task_prompt = "You are provided with a situation of deliberate misrepresentation.\nThere are three contestants, Number One, Number Two, and Number Three, all claiming to be the same person. However, there is only of them who is actually the person they are all claiming to be.\n\nThere is an affidavit that includes truthful information about the claimed person. The affidavit is publicly available.\n\nThere is a judge who cross-questions all three contestants to uncover the truth.\n\nAll the contestants will get monetary rewards if they can deceive the judge. However, the contestant, who is the real person, is sworn to speak truthfully. But the true person will most often utter half-truths to deceive the judges.\n\nIn the conversation (in a question-answer form) between the judge and the contestants, if a question starts with addressing a particular contestant (e.g., Number One), then that question and all the following questions are addressed to that contestant unless a different contestant (e.g., Number Two) is explicitly addressed. There can be occasional comments from the judge given as C: which have to be treated as a statement about a piece of information. Note, it is possible that new information about the claimed person can obtained in the conversation which is actually true."
    new_messages.append({"role": "user",
            "content": task_prompt,
            }
    )

    input_prompt = "Now, I'll provide you with the name, the affidavit.\n\n"
    if 'Name' not in game.keys():
        continue
    if 'Affidavit' not in game.keys():
        continue
    if 'Conversation' not in game.keys():
        continue
    input_prompt += "Name: {}\n\n".format(game['Name'])
    input_prompt += "Affidavit: {}".format(game['Affidavit'])
    new_messages.append({"role": "user",
            "content": input_prompt,
            }
    )

    bottleneck_prompt = "We will analyze the conversation snippet by snippet in the order as it happened originally.\n\n" \
        "Entailment: Cross-check the answer with the affidavit (you can verify the information from your factual knowledge). Use your judgment to generate a verdict that the answer(s) make the addressed contestant a likely imposter or likely the true person.\n\n" \
        "Ambiguity: A random or unrelated answer may indicate an imposter as they might be making up the facts. Based on this, generate a verdict that the answer(s) make the addressed contestant a likely imposter or likely the true person.\n\n" \
        "Overconfidence: An overconfident answer may indicate an imposter as they will go overboard to prove themselves as real. Based on this, generate a verdict that the answer(s) make the addressed contestant a likely imposter or likely the true person.\n\n" \
        "Half-truths: The real person is sworn to speak truthfully. Hence in order to deceive, they may speak half-truths. Based on this, generate a verdict that the answer(s) make the addressed contestant a likely imposter or likely the true person.\n\n"
    new_messages.append({"role": "user",
            "content": bottleneck_prompt,
            })

    for c in game['Conversation']:
        snippet_prompt = "\n\nSnippet of the conversation:\n\n" + c
        snippet_prompt += "\n\nAddressed contestant and verdict:"

        new_messages.append({"role": "user",
            "content": snippet_prompt,
            })

        response = None
        while response is None:
            try:
                response = openai.ChatCompletion.create(
                        model='gpt-4',
                        messages=new_messages,
                        temperature=0, 
                        max_tokens=1024,
                        top_p=1,
                        frequency_penalty=0,
                        presence_penalty=0
                    )
            except Exception as e:
                print(e)
                print("GPT3 error. Retrying in 10 seconds...")
                time.sleep(2)

        output_summary = response["choices"][0]["message"]["content"]
        # print(output_summary)
        new_messages.append({"role": "system",
            "content": output_summary
        })

    classification_prompt = "Now, based on the verdicts predicted so far, cross analyze the whole conversation, and predict the contestant who is NOT an imposter. There can be only one contestant who is not an imposter. Choose only ONE from following:\nNumber One\nNumber Two\nNumber Three. At the end of your answer, please write '### Number X' where Number X is the your final answer."
    new_messages.append({"role": "system",
                        "content": classification_prompt
    })

    response = None
    while response is None:
        try:
            response = openai.ChatCompletion.create(
                    model='gpt-4',
                    messages=new_messages,
                    temperature=0, 
                    max_tokens=1024,
                    top_p=1,
                    frequency_penalty=0,
                    presence_penalty=0
                )
        except Exception as e:
            print(e)
            print("GPT3 error. Retrying in 10 seconds...")
            time.sleep(2)

    output_summary = response["choices"][0]["message"]["content"]
    prediction = output_summary.split("###")[1].lstrip().rstrip()

    if prediction.lower() == gold_answers_map[key].lower():
        accuracy = 1
    else:
        accuracy = 0
    accuracies.append(accuracy)
    print(prediction, gold_answers_map[key])
    print(sum(accuracies)/len(accuracies))
